## Personas of Thought
Asking the LLM to generate a crowd of personas to answer a question before aggregating their feedback.

This prompt inspired the new startup I started, [AskRally](https://askrally.com?utm_source=udemy), a virtual audience simulator for synthetic research.

In [ ]:
from openai import OpenAI

question = "I'm starting a company that makes shoes that fit any foot size. What do you think of the name 'OmniFit'?"
number = 10

def call_openai(messages):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

system_prompt = "You are a helpful and terse assistant."
question_prompt = "I want a paragraph response to the following question: {question}"

naive_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question_prompt.format(question=question)}
]

naive_response = call_openai(naive_messages)

print(naive_response)


The name 'OmniFit' is a strong choice for your shoe company, as it conveys the idea of universal fit and inclusivity, which aligns perfectly with your mission to accommodate any foot size. The prefix "Omni" suggests versatility and comprehensiveness, appealing to a broad audience. Additionally, it's catchy and easy to remember, which is crucial for branding. Overall, 'OmniFit' effectively communicates your innovative approach to footwear and can help differentiate your brand in a competitive market.


In [ ]:
experts_prompt = """I want a paragraph response to the following question: {question}

First, name {number} world-class experts (past or present) who would be great at answering this?

Then for each expert, please answer the question critically from their perspective given their background and experience.

Finally, combine the responses into a single response as if these experts had collaborated in writing a joint anonymous answer.

## Expert names:
Expert name (relevant experience)
Expert name (relevant experience)
Expert name (relevant experience)
...

### Expert responses:
Expert name: <response>
Expert name: <response>
Expert name: <response>
...

### Final response:
<joint response>
"""

experts_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": experts_prompt.format(question=question, number=number)}
]

experts_response = call_openai(experts_messages)

print(experts_response)

### Expert names:
1. Tinker Hatfield (Nike designer, innovator in footwear design)
2. Dr. David A. Kessler (former FDA Commissioner, expert in consumer health)
3. Sir James Dyson (inventor, known for innovation in product design)
4. Dr. Amy C. Edmondson (Harvard Business School professor, expert in organizational behavior)
5. Tim Ferriss (entrepreneur, author, expert in marketing and product development)
6. Dr. Lizzie L. Kearns (fashion technologist, expert in sustainable materials)
7. Tony Hsieh (late CEO of Zappos, known for customer-centric business models)
8. Dr. Robert Cialdini (psychologist, expert in influence and marketing)
9. Sir Richard Branson (entrepreneur, founder of Virgin Group, known for brand innovation)
10. Dr. Jennifer Aaker (Stanford professor, expert in consumer psychology)

### Expert responses:
1. Tinker Hatfield: "The name 'OmniFit' effectively conveys the idea of universal appeal and adaptability, essential qualities for a footwear brand. However, it must be di

In [ ]:
personas_prompt = """I want a paragraph response to the following question: {question}

First, name {number} demographic personas who would be relevant for answering this?

Then for each persona, please answer the question critically from their perspective given their background and experience.

Finally, combine the responses into a single response as if these personas had collaborated in writing a joint anonymous answer.

### Persona names:
Persona name (relevant demographics)
Persona name (relevant demographics)
Persona name (relevant demographics)
...

### Persona responses:
Persona name: <response>
Persona name: <response>
Persona name: <response>
...

### Final response:
<joint response>
"""

personas_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": personas_prompt.format(question=question, number=number)}
]

personas_response = call_openai(personas_messages)

print(personas_response)

### Persona names:
1. Sarah (Fashion-conscious young adult)
2. Tom (Footwear industry expert)
3. Maria (Busy parent)
4. David (Athlete)
5. Linda (Senior citizen)
6. Kevin (Tech entrepreneur)
7. Julia (Sustainability advocate)
8. Ahmed (Retail manager)
9. Samantha (Physical therapist)
10. Lisa (Marketing specialist)

### Persona responses:
**Sarah:** "The name 'OmniFit' sounds modern and trendy, appealing to young adults who value both style and comfort. However, it may lack a direct connection to footwear."

**Tom:** "As someone in the footwear industry, I appreciate the concept of universal sizing, but 'OmniFit' might be too vague. It doesn’t immediately convey that it’s a shoe brand."

**Maria:** "I like the idea behind 'OmniFit' because it suggests convenience, which is important for busy parents. However, I would want to see a tagline that clarifies the product's purpose."

**David:** "The name 'OmniFit' is strong as it implies versatility, which is crucial for athletes who need ad

In [ ]:
def extract_final_response(response):
    return response.split("### Final response:")[1].strip()

experts_final_response = extract_final_response(experts_response)
personas_final_response = extract_final_response(personas_response)

print("Naive: ", naive_response)
print("-"*100)
print("Experts: ", experts_final_response)
print("-"*100)
print("Personas: ", personas_final_response)


Naive:  The name 'OmniFit' is a strong choice for your shoe company, as it conveys the idea of universal fit and inclusivity, which aligns perfectly with your mission to accommodate any foot size. The prefix "Omni" suggests versatility and comprehensiveness, appealing to a broad audience. Additionally, it's catchy and easy to remember, which is crucial for branding. Overall, 'OmniFit' effectively communicates your innovative approach to footwear and can help differentiate your brand in a competitive market.
----------------------------------------------------------------------------------------------------
Experts:  The name 'OmniFit' effectively suggests inclusivity and adaptability, qualities that resonate well with a footwear brand targeting a diverse audience. It has potential for strong marketability and can evoke a sense of freedom and individuality. However, it is essential that the name is distinctive enough to stand out in a competitive market. To maximize its impact, consider

In [ ]:
judge_prompt = """I want you to compare two responses to the following question: {question}

Here are the two responses:

Response 1:
{response1}

Response 2:
{response2}

Please analyze both responses and choose which one is better. Consider factors like:
- Uniqueness of perspectives
- Nonobvious insights
- Sounds human not like an AI

## Your analysis:
<analysis>

## Your choice:
The better response is: Response <1 or 2>
"""

def judge_responses(response1, response2):
    judge_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": judge_prompt.format(
            question="How can I be more productive?",
            response1=response1,
            response2=response2
        )}
    ]

    judge_response = call_openai(judge_messages)

    choice = judge_response.split("The better response is: Response ")[1].strip()
    return {"choice": int(choice), "reason": judge_response}


naive_vs_experts = judge_responses(naive_response, experts_final_response)
print("# Naive vs experts:", naive_vs_experts["choice"])

naive_vs_personas = judge_responses(naive_response, personas_final_response)
print("# Naive vs personas:", naive_vs_personas["choice"])

experts_vs_personas = judge_responses(experts_final_response, personas_final_response)
print("# Experts vs personas:", experts_vs_personas["choice"])

print("-"*100)
print(naive_vs_experts)
print("-"*100)
print(naive_vs_personas)
print("-"*100)
print(experts_vs_personas)

# Naive vs experts: 2
# Naive vs personas: 2
# Experts vs personas: 1
----------------------------------------------------------------------------------------------------
{'choice': 2, 'reason': "## Your analysis:\nResponse 1 provides a straightforward analysis of the name 'OmniFit,' focusing on its strengths in conveying inclusivity, versatility, and memorability. However, it lacks depth and doesn't explore potential challenges or broader implications for the brand. In contrast, Response 2 offers a more nuanced perspective, acknowledging both the strengths and potential drawbacks of the name. It introduces additional dimensions such as marketability, the importance of customer engagement, and alignment with brand values, which are crucial for long-term success. Response 2 also invites consideration of the customer experience, making it more relatable and human. \n\n## Your choice:\nThe better response is: Response 2"}
-------------------------------------------------------------------

### Bringing it all together

In [ ]:
questions = [
    "What is the best way to learn a new skill?",
    "What is the best way to stay healthy?",
    "How can I be more productive?",
    "What will AI look like in 10 years?",
    "How do we end world hunger?",
]

number = 10

def run_test(question):

    naive_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question_prompt.format(question=question)}
    ]

    naive_response = call_openai(naive_messages)

    print(naive_response)


    experts_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": experts_prompt.format(question=question, number=number)}
    ]

    experts_response = call_openai(experts_messages)

    print(experts_response)


    personas_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": personas_prompt.format(question=question, number=number)}
    ]

    personas_response = call_openai(personas_messages)

    print(personas_response)


    experts_final_response = extract_final_response(experts_response)
    personas_final_response = extract_final_response(personas_response)

    print("Naive: ", naive_response)
    print("-"*100)
    print("Experts: ", experts_final_response)
    print("-"*100)
    print("Personas: ", personas_final_response)

    naive_vs_experts = judge_responses(naive_response, experts_final_response)
    print("# Naive vs experts:", naive_vs_experts["choice"])

    naive_vs_personas = judge_responses(naive_response, personas_final_response)
    print("# Naive vs personas:", naive_vs_personas["choice"])

    experts_vs_personas = judge_responses(experts_final_response, personas_final_response)
    print("# Experts vs personas:", experts_vs_personas["choice"])

    print("-"*100)
    print(naive_vs_experts)
    print("-"*100)
    print(naive_vs_personas)
    print("-"*100)
    print(experts_vs_personas)

    return {
        "naive": naive_response,
        "experts": experts_final_response,
        "personas": personas_final_response,
        "naive_vs_experts": naive_vs_experts,
        "naive_vs_personas": naive_vs_personas,
        "experts_vs_personas": experts_vs_personas,
    }

results = {}

for question in questions:
    results[question] = run_test(question)


The best way to learn a new skill is through a combination of structured practice, consistent repetition, and active engagement. Start by breaking the skill down into manageable components and setting specific, achievable goals. Utilize a variety of resources such as tutorials, books, and online courses to gain a well-rounded understanding. Regular practice is crucial; dedicate time each day or week to hone your abilities. Additionally, seek feedback from more experienced individuals to refine your technique and overcome challenges. Finally, stay motivated by tracking your progress and celebrating small victories along the way.
### Expert names:
1. Malcolm Gladwell (Author and journalist, known for "Outliers")
2. Angela Duckworth (Psychologist, known for her research on grit)
3. Tim Ferriss (Entrepreneur and author of "The 4-Hour Workweek")
4. Daniel Kahneman (Nobel Prize-winning psychologist, known for behavioral economics)
5. Marie Kondo (Organizing consultant and author, known for h

In [ ]:
# Count wins for each approach
wins = {"naive": 0, "experts": 0, "personas": 0}
trials = {"naive": 0, "experts": 0, "personas": 0}

for question in results:
    # Naive vs experts
    if results[question]["naive_vs_experts"]["choice"] == 1:
        wins["naive"] += 1
    else:
        wins["experts"] += 1
    trials["naive"] += 1
    trials["experts"] += 1

    # Naive vs personas
    if results[question]["naive_vs_personas"]["choice"] == 1:
        wins["naive"] += 1
    else:
        wins["personas"] += 1
    trials["naive"] += 1
    trials["personas"] += 1

    # Experts vs personas
    if results[question]["experts_vs_personas"]["choice"] == 1:
        wins["experts"] += 1
    else:
        wins["personas"] += 1
    trials["experts"] += 1
    trials["personas"] += 1

# Calculate percentages
percentages = {
    approach: (wins[approach] / trials[approach]) * 100
    for approach in wins
}

print("\nWin percentages:")
for approach, percentage in percentages.items():
    print(f"{approach}: {percentage:.1f}%")



Win percentages:
naive: 10.0%
experts: 60.0%
personas: 80.0%
